In [1]:
#dependencies
import pandas as pd
import numpy as np

In [2]:
#schools file
schools = "Resources/schools_complete.csv"
schools_df = pd.read_csv(schools)

In [3]:
#students file
students = "Resources/students_complete.csv"
students_df = pd.read_csv(students)

Table 1. Merged data files

In [4]:
#merge data frames
combined_df = pd.merge(students_df, schools_df)

#fix column names
combined_df.rename(columns={"student_name":"Student", "school_name":"School", "reading_score":"Reading score", "math_score":"Math score", "type":"Type", "size":"Size", "gender":"Gender", "grade":"Grade", "budget":"Budget"}, inplace=True)
#("Student ID" column duplicates index column, but used below; yes, there are other ways.)

#change "grade" to numeric
combined_df.Grade = combined_df.Grade.str.replace('th','')
combined_df.Grade = combined_df.Grade.apply(pd.to_numeric)

combined_df

,Student ID,Student,Gender,Grade,School,Reading score,Math score,School ID,Type,Size,Budget
0,0,Paul Bradley,M,9,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,99,90,14,Charter,1635,1043130


Table 2. District Summary

In [5]:
#count schools and students
total_schools = len(combined_df["School ID"].unique())
total_students = len(combined_df["Student ID"].unique())

#add budgets
budgets_by_school_df = combined_df[["School","Budget"]].drop_duplicates()
total_budget = sum(budgets_by_school_df["Budget"])

#average math and reading scores
reading_average = round(combined_df["Reading score"].mean())
math_average = round(combined_df["Math score"].mean())

#find percentage of students passing
percentage_passed_reading = round((((combined_df["Reading score"]>=70).sum())/total_students)*100)
percentage_passed_math = round((((combined_df["Math score"]>=70).sum())/total_students)*100)
percentage_passed_overall = round(((((combined_df["Math score"]>=70) & (combined_df["Reading score"]>=70)).sum())/total_students) * 100)

#set up chart
district_df = pd.DataFrame({"Total schools": total_schools, "Total students": total_students, "Total budget": total_budget, \
                 "Mean reading score": reading_average, "Mean math score": math_average, \
                 "% Passing reading": percentage_passed_reading,"% Passing math": percentage_passed_math,\
                 "% Passing overall": percentage_passed_overall}, index=["Summary"])

format={"Total students":'{0:,.0f}', "Total budget":'${0:,.0f}', "Mean math score": '{:.1f}',"Mean reading score": '{:.1f}', "% Passing reading": '{:.0f}%', "% Passing math": '{:.0f}%', "% Passing overall": '{:.0f}%'}
district_df = district_df.style.format(format)

district_df

,Total schools,Total students,Total budget,Mean reading score,Mean math score,% Passing reading,% Passing math,% Passing overall
Summary,15,"39,170","$24,649,428",82.0,79.0,86%,75%,65%


Table 3. School Summary

In [6]:
#Reference
#https://stackoverflow.com/questions/44635626/rename-result-columns-from-pandas-aggregation-futurewarning-using-a-dict-with

def means_agg(x):
    names = {
        "Mean reading score": x["Reading score"].mean(),
        "Mean math score": x["Math score"].mean(),
    }
    return pd.Series(names, index=["Mean reading score", "Mean math score"])

grouped_by_school_df = combined_df.groupby(["School ID"]).apply(means_agg)

#Count passing students (math, reading, both)
number_passing_math = combined_df[combined_df['Math score'] >=70]
number_passing_math_grouped = number_passing_math.groupby('School ID')['Math score'].size().reset_index(name='Number passing math')
number_passing_math_grouped.set_index("School ID")

number_passing_reading = combined_df[combined_df['Reading score'] >=70]
number_passing_reading_grouped = number_passing_reading.groupby('School ID')['Reading score'].size().reset_index(name='Number passing reading')
number_passing_reading_grouped.set_index("School ID")

number_passing_overall = combined_df[(combined_df["Math score"]>=70) & (combined_df["Reading score"]>=70)]
number_passing_overall_grouped = number_passing_overall.groupby('School ID')['Reading score'].size().reset_index(name='Number passing overall')
number_passing_overall_grouped.set_index("School ID")

#How about passing reading but not math, and vice versa?
number_passing_only_reading = combined_df[(combined_df["Math score"]<70) & (combined_df["Reading score"]>=70)]
number_passing_only_reading_grouped = number_passing_only_reading.groupby('School ID')['Reading score'].size().reset_index(name='Number passing only reading')
number_passing_only_reading_grouped.set_index("School ID")

number_passing_only_math = combined_df[(combined_df["Math score"]>=70) & (combined_df["Reading score"]<70)]
number_passing_only_math_grouped = number_passing_only_math.groupby('School ID')['Reading score'].size().reset_index(name='Number passing only math')
number_passing_only_math_grouped.set_index("School ID")

#Join datafiles
combined_df.set_index("School ID")

budgets_by_school_df = combined_df[["School ID","School", "Type", "Size", "Budget"]].drop_duplicates()
budgets_by_school_df.set_index("School ID", inplace=True)
budgets_by_school_df["Per-student budget"] = budgets_by_school_df["Budget"]/budgets_by_school_df["Size"]

full_summary_by_school_df = budgets_by_school_df.join(grouped_by_school_df, how="outer")

full_summary_by_school_df = full_summary_by_school_df.join(number_passing_reading_grouped['Number passing reading'], how="outer")
full_summary_by_school_df = full_summary_by_school_df.join(number_passing_math_grouped['Number passing math'], how="outer")
full_summary_by_school_df = full_summary_by_school_df.join(number_passing_overall_grouped['Number passing overall'], how="outer")
full_summary_by_school_df = full_summary_by_school_df.join(number_passing_only_reading_grouped['Number passing only reading'], how="outer")
full_summary_by_school_df = full_summary_by_school_df.join(number_passing_only_math_grouped['Number passing only math'], how="outer")

#Format, remove unwanted columns (kept "only reading" and "only math" out of curiosity)
full_summary_by_school_df["% Passing reading"] = round((full_summary_by_school_df["Number passing reading"]/full_summary_by_school_df["Size"])*100)
full_summary_by_school_df["% Passing math"] = round((full_summary_by_school_df["Number passing math"]/full_summary_by_school_df["Size"])*100)
full_summary_by_school_df["% Passing overall"] = round((full_summary_by_school_df["Number passing overall"]/full_summary_by_school_df["Size"])*100)
full_summary_by_school_df["% Passing only reading"] = round((full_summary_by_school_df["Number passing only reading"]/full_summary_by_school_df["Size"])*100)
full_summary_by_school_df["% Passing only math"] = round((full_summary_by_school_df["Number passing only math"]/full_summary_by_school_df["Size"])*100)

format={"Budget":'${0:,.0f}', "Per-student budget":'${0:,.2f}', "Mean math score": '{:.1f}',"Mean reading score": '{:.1f}', "% Passing reading": '{:.0f}%', "% Passing math": '{:.0f}%', "% Passing overall": '{:.0f}%', "% Passing only reading": '{:.0f}%', "% Passing only math": '{:.0f}%'}
short_summary_by_school_df = full_summary_by_school_df.drop(columns=['Number passing reading','Number passing math', 'Number passing overall', 'Number passing only reading','Number passing only math'])
short_summary_by_school_pretty_df = short_summary_by_school_df.style.format(format)

short_summary_by_school_pretty_df

,School,Type,Size,Budget,Per-student budget,Mean reading score,Mean math score,% Passing reading,% Passing math,% Passing overall,% Passing only reading,% Passing only math
School ID,,,,,,,,,,,,
0,Huang High School,District,2917,"$1,910,635",$655.00,81.2,76.6,81%,66%,54%,28%,12%
1,Figueroa High School,District,2949,"$1,884,411",$639.00,81.2,76.7,81%,66%,53%,28%,13%
2,Shelton High School,Charter,1761,"$1,056,600",$600.00,83.7,83.4,96%,94%,90%,6%,4%
3,Hernandez High School,District,4635,"$3,022,020",$652.00,80.9,77.3,81%,67%,54%,27%,13%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.8,83.4,97%,93%,91%,7%,3%
5,Wilson High School,Charter,2283,"$1,319,574",$578.00,84.0,83.3,97%,94%,91%,6%,3%
6,Cabrera High School,Charter,1858,"$1,081,356",$582.00,84.0,83.1,97%,94%,91%,6%,3%
7,Bailey High School,District,4976,"$3,124,928",$628.00,81.0,77.0,82%,67%,55%,27%,12%
8,Holden High School,Charter,427,"$248,087",$581.00,83.8,83.8,96%,93%,89%,7%,3%


Table 4. Top Performing Schools (by % overall passing)

In [7]:
five_best_df = short_summary_by_school_df.nlargest(5, "% Passing overall")

five_best_pretty_df = five_best_df.style.format(format)

five_best_pretty_df

,School,Type,Size,Budget,Per-student budget,Mean reading score,Mean math score,% Passing reading,% Passing math,% Passing overall,% Passing only reading,% Passing only math
School ID,,,,,,,,,,,,
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.8,83.4,97%,93%,91%,7%,3%
5,Wilson High School,Charter,2283,"$1,319,574",$578.00,84.0,83.3,97%,94%,91%,6%,3%
6,Cabrera High School,Charter,1858,"$1,081,356",$582.00,84.0,83.1,97%,94%,91%,6%,3%
9,Pena High School,Charter,962,"$585,858",$609.00,84.0,83.8,96%,95%,91%,5%,4%
14,Thomas High School,Charter,1635,"$1,043,130",$638.00,83.8,83.4,97%,93%,91%,6%,2%


Table 5. Bottom Performing Schools (by % overall passing)

In [8]:
five_lowest_df = short_summary_by_school_df.nsmallest(5, "% Passing overall")

five_lowest_pretty_df = five_lowest_df.style.format(format)

five_lowest_pretty_df

,School,Type,Size,Budget,Per-student budget,Mean reading score,Mean math score,% Passing reading,% Passing math,% Passing overall,% Passing only reading,% Passing only math
School ID,,,,,,,,,,,,
1,Figueroa High School,District,2949,"$1,884,411",$639.00,81.2,76.7,81%,66%,53%,28%,13%
11,Rodriguez High School,District,3999,"$2,547,363",$637.00,80.7,76.8,80%,66%,53%,27%,13%
0,Huang High School,District,2917,"$1,910,635",$655.00,81.2,76.6,81%,66%,54%,28%,12%
3,Hernandez High School,District,4635,"$3,022,020",$652.00,80.9,77.3,81%,67%,54%,27%,13%
12,Johnson High School,District,4761,"$3,094,650",$650.00,81.0,77.1,81%,66%,54%,28%,13%


Table 6. Math Scores by School and Grade

In [9]:
def means_agg_2m(x):
    names = {
        "Mean math score": x["Math score"].mean()
    }
    return pd.Series(names, index=["Mean math score"])

grouped_by_grade_for_math_score_df = combined_df.groupby(["School ID", "School","Grade"]).apply(means_agg_2m)

grouped_by_grade_for_math_score_pretty_df = grouped_by_grade_for_math_score_df.style.format({"Mean math score": '{:.1f}'})

grouped_by_grade_for_math_score_pretty_df

Table 7. Reading Scores by School and Grade

In [10]:
def means_agg_2r(x):
    names = {
        "Mean reading score": x["Reading score"].mean()
    }
    return pd.Series(names, index=["Mean reading score"])

grouped_by_grade_for_reading_score_df = combined_df.groupby(["School ID", "School","Grade"]).apply(means_agg_2r)

grouped_by_grade_for_reading_score_pretty_df = grouped_by_grade_for_reading_score_df.style.format({"Mean reading score": '{:.1f}'})

grouped_by_grade_for_reading_score_pretty_df

Table 8. Scores by School Spending

In [11]:
index=["Mean reading score", "Mean math score", "Mean % passing reading", "Mean % passing math", "Mean % passing overall"]
format2={"Mean reading score": '{:.1f}',"Mean math score": '{:.1f}', "Mean % passing reading": '{:.0f}%', "Mean % passing math": '{:.0f}%', "Mean % passing overall": '{:.0f}%'}
short_summary_by_school_df['Per-student budget range'] = pd.cut(short_summary_by_school_df["Per-student budget"], 4, labels = ["1 ($578 - 592]", "2 ($592 - 628]", "3 ($628 - 642]", "4 ($642 - 655]"])

def means_agg_3(x):
    names = {
        "Mean reading score": x['Mean reading score'].mean(),
        "Mean math score": x['Mean math score'].mean(),
        "Mean % passing reading": x['% Passing reading'].mean(),
        "Mean % passing math": x['% Passing math'].mean(),
        "Mean % passing overall": x['% Passing overall'].mean()
    }
    return pd.Series(names,index)

grouped_by_per_student_budget_df = short_summary_by_school_df.groupby(['Per-student budget range']).apply(means_agg_3)

grouped_by_per_student_budget_df = grouped_by_per_student_budget_df.rename_axis(index='Per-student budget category')
grouped_by_per_student_budget_prettier_df = grouped_by_per_student_budget_df.style.format(format2)

grouped_by_per_student_budget_prettier_df

,Mean reading score,Mean math score,Mean % passing reading,Mean % passing math,Mean % passing overall
Per-student budget category,,,,,
1 ($578 - 592],83.9,83.5,97%,94%,90%
2 ($592 - 628],83.9,83.6,96%,94%,90%
3 ($628 - 642],82.4,80.2,90%,80%,73%
4 ($642 - 655],81.4,77.9,83%,70%,59%


Table 9. Scores by School Size

In [12]:
short_summary_by_school_df['School size'] = pd.cut(short_summary_by_school_df["Size"], 3, labels = ["Small (422 - 1943]", "Medium (1943 - 3460]", "Large (3460 - 4976]"])
grouped_by_school_size_df = short_summary_by_school_df.groupby(['School size']).apply(means_agg_3)

grouped_by_school_size_df = grouped_by_school_size_df.rename_axis(index='School size')
grouped_by_school_size_pretty_df = grouped_by_school_size_df.style.format(format2)

grouped_by_school_size_pretty_df

,Mean reading score,Mean math score,Mean % passing reading,Mean % passing math,Mean % passing overall
School size,,,,,
Small (422 - 1943],83.9,83.5,97%,94%,90%
Medium (1943 - 3460],81.8,78.4,84%,74%,63%
Large (3460 - 4976],80.9,77.1,81%,66%,54%


Table 10. Scores by School Type

In [13]:
grouped_by_type_df = short_summary_by_school_df.groupby(['Type']).apply(means_agg_3)

grouped_by_type_pretty_df = grouped_by_type_df.style.format(format2)

grouped_by_type_pretty_df

,Mean reading score,Mean math score,Mean % passing reading,Mean % passing math,Mean % passing overall
Type,,,,,
Charter,83.9,83.5,97%,94%,90%
District,81.0,77.0,81%,67%,54%
